# Bloom Filters & EM Algorithm
+ Core Bloom filter routines.
+ Unsupervised EM algorithm.
+ Evaluation with hard match, Dice and Damerau Levenshtein weights.
+ Reference: Herzog, Scheuren, Winkler, "Data Quality and Record Linkage Techniques", Springer, 2007.

In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import re, math
import hashlib
from scipy.special import factorial
from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
tqdm.pandas()

from jellyfish import damerau_levenshtein_distance

import recordlinkage as rl
from recordlinkage.datasets import load_febrl2, load_febrl4, binary_vectors
from pandas.api.types import is_string_dtype



## Create Fake Data Sets
- These tables would be the raw CSV data provided by an agency after read_csv().
- All fields will be forced to have string type.
- Missing entries will be loaded as NaN.

In [2]:
dfA, dfB = load_febrl4()

In [3]:
def generate_csv_table(tbl):
    return pd.DataFrame({
        'FirstName': tbl.given_name,
        'LastName': tbl.surname,
        'BirthYear': tbl.date_of_birth.str[0:4],
        'BirthMonth': tbl.date_of_birth.str[4:6],
        'BirthDay': tbl.date_of_birth.str[6:8]
    }).fillna('')
    
    

In [4]:
pTxtA = generate_csv_table(dfA)
pTxtB = generate_csv_table(dfB)

## Bloom Filter

In [5]:
def bloom_filter(plnTxt, salt, qGrmLen, filterLen):
    
    plnTxt = f'_{plnTxt}_'
    saltStr = f'{salt:x}'
    bloomFilter = 0
    
    for i in range(len(plnTxt)-qGrmLen+1):
        byteStr = (saltStr + plnTxt[i:i+qGrmLen]).encode('utf-8','replace')
    
        idxMd5 = int( hashlib.md5(byteStr).hexdigest(), 16) % filterLen
        idxSha = int( hashlib.sha256(byteStr).hexdigest(), 16) % filterLen
        
        bloomFilter |= 1 << idxMd5
        bloomFilter |= 1 << idxSha
        
    return bloomFilter
        

In [6]:
bloomPrms = [ 
    { 'fLen': 32, 'qLen': 2 }
]                        

In [7]:
def gen_bloom_values(tbl,salt):

    def construct_row(row,salt):

        bloomVals = {}
        for bloomPrm in bloomPrms:
            for index, val in row.items():
                colStr = f"{index}Bf{bloomPrm['fLen']}Q{bloomPrm['qLen']}"
                colVal = bloom_filter(val,salt,bloomPrm['qLen'],bloomPrm['fLen'])
                bloomVals[colStr] = colVal

        return pd.concat([ row, pd.Series(bloomVals) ])
    
    return tbl.apply(construct_row,salt=salt,axis=1)
    

In [8]:
dsetA = gen_bloom_values(pTxtA,salt=0)
dsetB = gen_bloom_values(pTxtB,salt=0)

## EM Algorithm

## $\gamma$
- This is represents the result of a hard match where $\gamma_i^j = 1$ if the $i$th data feature matches for the $j$th record pair.
- In general, the record pairs are created by examining the full cross product of our populations $A$ and $B$ so that $N = |A \times B| = |A| \times |B|$.  The number of records may be reduced if blocking is used.

In [9]:
ftrCols = [ 'FirstName', 'LastName', 'BirthYear', 'BirthMonth', 'BirthDay' ]
nFtr = len(ftrCols)  # N

In [10]:
def gen_gamma(dsetA,dsetB,ftrCols):

    yearBlockRange = 5
    
    nRec = len(dsetA.index) * len(dsetB.index) 
    
    gamma = np.zeros((nRec,len(ftrCols)),dtype='int8')
    recKeys = np.empty((nRec,2),dtype='<U30')
    
    idxRec = 0
    pBar = tqdm(total=nRec)
    for idxA, rowA in dsetA.iterrows():
        for idxB, rowB in dsetB.iterrows():

            if rowA.BirthYear != '' and rowB.BirthYear != '' and np.abs(int(rowA.BirthYear) - int(rowB.BirthYear)) < yearBlockRange:
                recKeys[idxRec,:] = [ idxA, idxB ]
                gamma[idxRec,:] = (rowA[ftrCols] == rowB[ftrCols]).to_numpy()

                idxRec += 1
            pBar.update()

    pBar.close()

    return (gamma[0:idxRec,:],recKeys[0:idxRec,:])

In [11]:
(gamma,recKeys) = gen_gamma(dsetA,dsetB,ftrCols)
nRec = gamma.shape[0]

  0%|          | 0/25000000 [00:00<?, ?it/s]

In [31]:
gTruth = np.empty(nRec,dtype=int)

for j in range(nRec):
    gTruth[j] = re.search('-\d+-',recKeys[j,0])[0][1:-1] == re.search('-\d+-',recKeys[j,1])[0][1:-1]

## $m$ and $u$ Probabilities
- $m = [ m_1, \ldots m_N ]$ where $m_i = P(\gamma_i =1 | r_j \in M)$.
- $u = [ u_1, \ldots u_N ]$ where $u_i = P(\gamma_i =1 | r_j \in U)$.
- These are overall probabilities determined for all matched and unmatched records.
- A good initial guess for $m_i$ is around 0.9.  Most records will match but a few won't due to typographic errors/intentional alterations.
- A good initial guess for $u_i$ would be around 0.1 for names, 1/12 for birth month and 1/31 for birth day.  These are the probabilities that different people just happen to share the same field.


In [13]:
m = np.array([ 0.9 ]*nFtr)
u = np.array([ 0.15 ]*nFtr)

## Membership in $M$ and $U$
- $M$ contains all records from the comparison of $A$ and $B$ that represent the same individual.  $U$ contains the record pairs that represent different individuals.
- The indicator function $g_j=1$ if the $j$th comparison record, $r_j \in M$.
- $p = |M|/N$

In [14]:
p = 1/5000 # Probability of being in M

## EM Algorithm
- The EM algorithm will iteratively refine $p$, $g$, $m$ and $u$.
- Our estimate of each indicator, $g_j$, is $p \cdot P(\gamma_j|r_j \in M)/\left[p \cdot P(\gamma_j|r_j \in M) + (1-p) \cdot P(\gamma_j|r_j \in U) \right]$.

In [15]:
g = np.zeros(nRec)

In [16]:
# Threshold for relative change in p below which the EM algorithm terminates.
pDiffThrsh = 0.1  

In [17]:
pOld = 1

while np.abs((p-pOld)/pOld) > pDiffThrsh:

    print('-- Update Weights --')
    
    for j in tqdm(range(nRec)):
        pGamInM = np.prod(gamma[j,:]*m + (1-gamma[j,:])*(1-m))
        pGamInU = np.prod(gamma[j,:]*u + (1-gamma[j,:])*(1-u))

        g[j] = p*pGamInM/(p*pGamInM + (1-p)*pGamInU)

    sumG = np.sum(g)
    sumGcmp = np.sum(1-g)
    
    for i in range(nFtr):
        m[i] = sum(g*gamma[:,i])/sumG
        u[i] = sum((1-g)*gamma[:,i])/sumGcmp

    pOld = p
    p = sumG/nRec
    
    print(f' New p: {p}, Old p: {pOld}, Rel Diff: {np.abs((p-pOld)/pOld)}\n')
        
        

-- Update Weights --


  0%|          | 0/2071742 [00:00<?, ?it/s]

 New p: 0.000661646983580876, Old p: 0.0002, Rel Diff: 2.30823491790438

-- Update Weights --


  0%|          | 0/2071742 [00:00<?, ?it/s]

 New p: 0.0017932769711960419, Old p: 0.000661646983580876, Rel Diff: 1.7103228998199485

-- Update Weights --


  0%|          | 0/2071742 [00:00<?, ?it/s]

 New p: 0.002002659894344032, Old p: 0.0017932769711960419, Rel Diff: 0.11675994646177844

-- Update Weights --


  0%|          | 0/2071742 [00:00<?, ?it/s]

 New p: 0.002077326973849734, Old p: 0.002002659894344032, Rel Diff: 0.03728395406358256



# Hard Classification
- Rather than use the 3 level classification system proposed by Fellegi and Sunter, take a binary maximum likelihood approach and choose either match or non-match based on which gives the maximum probability. 
- Use the log of the probabilities to simplify this calculation.

$$
\begin{array}{rl}
\log P(\gamma_j|r_j \in M)
& = \log\left[  \prod_i m_i^{\gamma_i^j}(1-m_i)^{1-\gamma_i^j}  \right] \\
& = \sum_i \gamma_i^j \log m_i + \sum_i (1-\gamma_i^j) \log (1-m_i) \\
\end{array}
$$




In [18]:
logM = np.log(m)
logMCmp = np.log(1-m)

logU = np.log(u)
logUCmp = np.log(1-u)

In [19]:
gHatHard = np.empty(nRec,dtype=int)
for j in tqdm(range(nRec)):
    wM = np.sum(gamma[j,:]*logM + (1-gamma[j,:])*logMCmp)
    wU = np.sum(gamma[j,:]*logU + (1-gamma[j,:])*logUCmp)
    gHatHard[j] = wM > wU
    
    

  0%|          | 0/2071742 [00:00<?, ?it/s]

### Discussion

In [20]:
def classification_summary(gHat,gTruth):
    tPos = np.sum(gHat*gTruth)
    fPos = np.sum(gHat*(1-gTruth))
    tNeg = np.sum((1-gHat)*(1-gTruth))
    fNeg = np.sum((1-gHat)*gTruth)
    
    print('Confusion Matrix:')
    print(f' True Pos: {tPos}/{tPos+fNeg}')
    print(f' False Neg: {fNeg}/{tPos+fNeg}')
    print(f' False Pos: {fPos}/{fPos+tNeg}')
    print(f' True Neg: {tNeg}/{fPos+tNeg}\n')
    
    print(f'Precision: {tPos/(tPos+fPos):.3f}')
    print(f'Recall: {tPos/(tPos+fNeg):.3f}')


In [21]:
classification_summary(gHatHard,gTruth)

Confusion Matrix:
 True Pos: 4550/4586
 False Neg: 36/4586
 False Pos: 848/2067156
 True Neg: 2066308/2067156

Precision: 0.843
Recall: 0.992


- This particular data set has a large number of false positives.
- Exploring the matches turn out to be individuals who have completely different names but the same birth date.

In [22]:
idx = np.argwhere(gHatHard*(1-gTruth))
recKeys[idx[0]]

array([['rec-383-org', 'rec-1574-dup-0']], dtype='<U30')

In [23]:
pTxtA.loc['rec-383-org']

FirstName        angus
LastName      mcgregor
BirthYear         1917
BirthMonth          04
BirthDay            09
Name: rec-383-org, dtype: object

In [24]:
pTxtB.loc['rec-1574-dup-0']

FirstName       zarran
LastName      mcsorley
BirthYear         1917
BirthMonth          04
BirthDay            09
Name: rec-1574-dup-0, dtype: object

- A large number of exact birthday matches are present in this dataset, either from deliberate lack of corruption or random matching of different individuals with the same birthdate.
- As a result, the m probabilities reward a birthday match more than an name match and a name mismatch is penalized less.
- The u probabilities recognize random birthday matches to a certain extent and assign a much higher value for birthday matches for two different individuals than for random name matches.

In [25]:
print(f'  m: {m}')
print(f'1-m: {1-m}')
print(f'  u: {u}')
print(f'1-m: {1-u}')

  m: [0.71240888 0.70969024 0.99899548 0.99153511 0.99611169]
1-m: [0.28759112 0.29030976 0.00100452 0.00846489 0.00388831]
  u: [0.00408187 0.00352935 0.11465814 0.0826882  0.03252481]
1-m: [0.99591813 0.99647065 0.88534186 0.9173118  0.96747519]


# Soft Classification
- Choose a metric (Levenshtein, Dice, etc.), $\Phi$, where $0 \leq \Phi \leq 1$.
- Let $w_{a,i} = \log(m_i/u_i)$ be the maximum *agreement weight*.
- Let $w_{d,i} = \log((1-m_i)/(1-u_i))$ be the maximum *disagreement weight*.
- The decision weight for feature $i$ is $\max[w_{a,i} - (w_{a_i}-w_{d,i})\Phi_i\alpha_i,w_{d,i}]$, where $\alpha_i$ is a constant controlling how quickly the decision weight decays with $\Phi_i$.
- The decision weights are summed and compared to a zero threshold.


In [26]:
logM = np.log(m)
logMCmp = np.log(1-m)

logU = np.log(u)
logUCmp = np.log(1-u)

wa = logM-logU
wd = logMCmp - logUCmp

In [27]:
def dice(bf1, bf2):
    return 2 * bin(bf1 & bf2).count("1") / ( bin(bf1).count("1") + bin(bf2).count("1") )

In [28]:
def norm_damleven(str1,str2):
    return 1 - damerau_levenshtein_distance(str1, str2) / np.max([len(str1), len(str2), 1])

In [29]:
ftrColsPTxt = [ 'FirstName', 'LastName', 'BirthYear', 'BirthMonth', 'BirthDay' ]
ftrColsBloom = [ 'FirstNameBf32Q2', 'LastNameBf32Q2', 'BirthYearBf32Q2', 'BirthMonthBf32Q2', 'BirthDayBf32Q2' ]

In [32]:
def soft_link(recKeys,ftrCols,alpha,phiFunc,dsetA,dsetB):

    nFtr = len(ftrCols)
    phi = np.zeros(nFtr)
    nRec = len(recKeys)

    gHat = np.empty(nRec,dtype=int)
    for j in tqdm(range(nRec)):

        idxA = recKeys[j,0]
        idxB = recKeys[j,1]
        
        for i in range(nFtr):
            phi[i] = phiFunc(dsetA.loc[idxA,ftrCols[i]],dsetB.loc[idxB,ftrCols[i]])

        w = np.sum(np.max(np.array([ wa - (wa-wd)*(1-phi)*alpha, wd ]),axis=0))
        gHat[j] = w > 0
        
    return gHat


In [33]:
alpha = np.array([4.5]*5)
gHatPTxt = soft_link(recKeys,ftrColsPTxt,alpha,norm_damleven,dsetA,dsetB)

  0%|          | 0/2071742 [00:00<?, ?it/s]

In [34]:
classification_summary(gHatPTxt,gTruth)

Confusion Matrix:
 True Pos: 4551/4586
 False Neg: 35/4586
 False Pos: 855/2067156
 True Neg: 2066301/2067156

Precision: 0.842
Recall: 0.992


In [35]:
gHatBloom = soft_link(recKeys,ftrColsBloom,alpha,dice,dsetA,dsetB)

  0%|          | 0/2071742 [00:00<?, ?it/s]

In [36]:
classification_summary(gHatBloom,gTruth)

Confusion Matrix:
 True Pos: 4553/4586
 False Neg: 33/4586
 False Pos: 1015/2067156
 True Neg: 2066141/2067156

Precision: 0.818
Recall: 0.993


In [37]:
def presentation_summary(gHat,gTruth):
    tPos = np.sum(gHat*gTruth)
    fPos = np.sum(gHat*(1-gTruth))
    tNeg = np.sum((1-gHat)*(1-gTruth))
    fNeg = np.sum((1-gHat)*gTruth)
    nRec = len(gHat)
        
    print(f'      Accuracy: {(tPos+tNeg)/nRec*100:.3f}%')
    print(f'     Precision: {tPos/(tPos+fPos)*100:.2f}%')
    print(f'   Sensitivity: {tPos/(tPos+fNeg)*100:.3f}%\n')
        
    print(f'      Correct Links: {tPos}')
    print(f'    Incorrect Links: {fPos}')
    print(f'       Missed Links: {fNeg}')
    print(f'  Correct Non-Links: {tNeg}\n')
    
    print(f'Number of Comparisons: {nRec}')

    

In [38]:
presentation_summary(gHatPTxt,gTruth)

      Accuracy: 99.957%
     Precision: 84.18%
   Sensitivity: 99.237%

      Correct Links: 4551
    Incorrect Links: 855
       Missed Links: 35
  Correct Non-Links: 2066301

Number of Comparisons: 2071742


In [39]:
presentation_summary(gHatBloom,gTruth)

      Accuracy: 99.949%
     Precision: 81.77%
   Sensitivity: 99.280%

      Correct Links: 4553
    Incorrect Links: 1015
       Missed Links: 33
  Correct Non-Links: 2066141

Number of Comparisons: 2071742
